Machine-Learning-Accelerated Catalytic
Activity Predictions of Transition Metal
Phthalocyanine Dual-Metal-Sites Catalysts for
CO2 Reduction


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [5]:
import pandas as pd
import itertools

# Original data
data = {
    'TM': ['Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'W', 'Ir', 'Pt', 'Au'],
    'edp': [10, 5, 5, 6, 7, 8, 10, 4, 5, 7, 8, 10, 8, 4, 7, 9, 10],
    'Hf.ox': [-11.39, -6.26, -5.49, -6.45, -5.6, -4.75, -3.26, -12.07, -7.71, -6.14, -4.84, -2.81, -0.9, -9.64, -6.51, -3.38, -0.4],
    'Nm': [1.54, 1.66, 1.55, 1.83, 1.88, 1.92, 1.9, 1.59, 2.16, 2.2, 2.28, 2.2, 1.93, 2.36, 2.2, 2.28, 2.54],
    'χ': [0.07554, 0.67584, -0.5, 0.153236, 0.66226, 1.15716, 1.23578, 0.9174, 0.7473, 1.04638, 1.14289, 0.56214, 1.30447, 0.81626, 1.56436, 2.1251, 2.30861],
    'Im': [6.8281, 6.7665, 7.43402, 7.9024, 7.881, 7.6398, 7.72638, 6.75885, 7.09243, 7.3605, 7.4589, 8.3369, 7.5762, 7.864, 8.967, 8.9587, 9.2255],
    'r': [187, 189, 197, 194, 192, 163, 140, 207, 209, 207, 195, 202, 172, 210, 202, 209, 166],
    'Nd': [2, 5, 5, 6, 7, 8, 10, 4, 5, 7, 8, 8, 10, 4, 7, 9, 10],
    'Q': [-1.72, -1.63, -1.61, -1.49, -1.54, -1.26, -1.22, -1.62, -1.67, -1.57, -1.24, -1.29, -0.95, -1.31, -1.23, -1.14, -1.03],
    'ΔGCOOH*': [-0.92, -1.15, -0.5, -1.23, -1.77, -0.03, 0.23, -1.38, -0.46, -0.69, -0.9, -0.39, -0.27, -1.85, -0.83, -0.74, 0.47],
    'ΔGMax': [1.41, 0.66, 0.94, 1.14, 0.64, 1.65, 1.77, 1.44, 0.85, 0.63, 1.81, 0.89, 0.51, 1.37, 0.59, 2.07, 2.44]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Generate all possible combinations of rows
combinations = list(itertools.combinations_with_replacement(df.index, 2))

# Create a new DataFrame for merged results
merged_data = []

for (i, j) in combinations:
    # Create a merged row
    merged_row = {
        'TM1': df.loc[i, 'TM'],
        'TM2': df.loc[j, 'TM']
    }
    # Add features for TM1
    for col in df.columns[1:]:
        merged_row[f'{col}_1'] = df.loc[i, col]

    # Add features for TM2
    for col in df.columns[1:]:
        merged_row[f'{col}_2'] = df.loc[j, col]

    # Append to merged data list
    merged_data.append(merged_row)

# Convert the merged data list into a DataFrame
merged_df = pd.DataFrame(merged_data)

# Show the first few rows of the merged DataFrame
merged_df.head()


,TM1,TM2,edp_1,Hf.ox_1,Nm_1,χ_1,Im_1,r_1,Nd_1,Q_1,...,edp_2,Hf.ox_2,Nm_2,χ_2,Im_2,r_2,Nd_2,Q_2,ΔGCOOH*_2,ΔGMax_2
0,Ti,Ti,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,10,-11.39,1.54,0.075540,6.82810,187,2,-1.72,-0.92,1.41
1,Ti,Cr,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,5,-6.26,1.66,0.675840,6.76650,189,5,-1.63,-1.15,0.66
2,Ti,Mn,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,5,-5.49,1.55,-0.500000,7.43402,197,5,-1.61,-0.50,0.94
3,Ti,Fe,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,6,-6.45,1.83,0.153236,7.90240,194,6,-1.49,-1.23,1.14
4,Ti,Co,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,7,-5.60,1.88,0.662260,7.88100,192,7,-1.54,-1.77,0.64


edp1:	Electron numbers (d+p orbital) of the TM1 atom
Hf.ox1:	Oxide formation enthalpy of the TM1 atom
Nm1:	Pauli electronegativity of the TM1 atom
χ1:	Electron affinity of the TM1 atom
Im1:	First ionization energy of the TM1 atom
r1:	Atomic radius of the TM1 atom
Nd1:	d Electron count of the TM1 atom
Q1:	Charge transfer from TM1 to Pc ring
ΔGCOOH*1:	Absorption energy of COOH* on the TM1 atom
ΔGMax1:	Free energy change of the CO2RR PDS on the TM1 atom
edp2:	Electron numbers (d+p orbital) of the side TM2 atom
Hf.ox2:	Oxide formation enthalpy of the side TM2 atom
Nm2:	Pauli electronegativity of the side TM2 atom
χ2:	Electron affinity of the side TM2 atom
Im2:	First ionization energy of the side TM2 atom
r2:	Atomic radius of the side TM2 atom
Nd2:	d Electron count of the side TM2 atom
Q2:	Charge transfer from TM22 to Pc ring
ΔGCOOH*2:	Absorption energy of COOH* on the side TM2 atom
ΔGMax2:	Free energy change of the CO2RR PDS on the side TM2 atom
UL (Target Variable):	Limiting potential


In [7]:
merged_df.shape

(153, 22)

In [8]:
merged_df.isnull().sum()

,0
TM1,0
TM2,0
edp_1,0
Hf.ox_1,0
Nm_1,0
χ_1,0
Im_1,0
r_1,0
Nd_1,0
Q_1,0


In [9]:
merged_df.duplicated().sum()

0

In [26]:
X=merged_df.iloc[ :,2: ]
X.head()

,edp_1,Hf.ox_1,Nm_1,χ_1,Im_1,r_1,Nd_1,Q_1,ΔGCOOH*_1,ΔGMax_1,...,Hf.ox_2,Nm_2,χ_2,Im_2,r_2,Nd_2,Q_2,ΔGCOOH*_2,ΔGMax_2,Target
0,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,-0.92,1.41,...,-11.39,1.54,0.075540,6.82810,187,2,-1.72,-0.92,1.41,1.30
1,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,-0.92,1.41,...,-6.26,1.66,0.675840,6.76650,189,5,-1.63,-1.15,0.66,1.05
2,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,-0.92,1.41,...,-5.49,1.55,-0.500000,7.43402,197,5,-1.61,-0.50,0.94,1.01
3,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,-0.92,1.41,...,-6.45,1.83,0.153236,7.90240,194,6,-1.49,-1.23,1.14,1.52
4,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,-0.92,1.41,...,-5.60,1.88,0.662260,7.88100,192,7,-1.54,-1.77,0.64,1.02


In [27]:
target_data = {
    'Ti': [1.3, 1.09, 1.1, 1.09, 0.87, 1.1, 1.06, 1.07, 1.22, 0.83, 0.74, 0.77, 0.68, 1.08, 0.81, 0.77, 0.75],
    'Cr': [1.05, 0.96, 0.87, 0.87, 0.82, 0.87, 0.83, 0.83, 1.23, 0.84, 0.74, 0.78, 0.68, 1.07, 0.83, 0.78, 0.74],
    'Mn': [1.01, 0.83, 0.7, 0.69, 0.66, 0.69, 0.66, 0.63, 1.23, 0.94, 0.81, 0.94, 0.71, 1.12, 0.94, 0.93, 0.82],
    'Fe': [1.52, 1.27, 1.27, 1.25, 1.01, 1.29, 1.24, 1.22, 1.23, 0.96, 0.76, 0.87, 0.68, 1.11, 0.92, 0.87, 0.75],
    'Co': [1.02, 0.83, 0.71, 0.71, 0.66, 0.71, 0.66, 0.66, 1.23, 0.93, 0.75, 0.84, 0.39, 1.08, 0.89, 0.84, 0.75],
    'Ni': [1.56, 1.42, 1.31, 1.27, 1.18, 1.29, 1.27, 1.24, 1.24, 0.96, 0.81, 0.94, 0.72, 1.12, 0.96, 0.94, 0.82],
    'Cu': [1.55, 1.18, 1.07, 1.05, 0.7, 1.08, 1.03, 1.02, 1.24, 0.96, 0.81, 0.94, 0.72, 1.12, 0.95, 0.94, 0.82],
    'Nb': [1.31, 1.11, 1.13, 1.11, 0.91, 1.12, 1.09, 1.07, 1.1, 0.69, 0.66, 0.7, 0.59, 0.97, 0.7, 0.7, 0.67],
    'Mo': [1.77, 1.57, 1.27, 1.2, 1.72, 1.2, 1.22, 1.15, 1.09, 0.62, 0.63, 0.68, 0.31, 0.93, 0.65, 0.65, 0.65],
    'Ru': [1.77, 1.72, 1.39, 1.35, 1.71, 1.37, 1.38, 1.32, 1.1, 0.75, 0.69, 0.76, 0.6, 0.97, 0.77, 0.76, 0.7],
    'Rh': [1.87, 1.92, 1.43, 1.39, 1.8, 1.43, 1.43, 1.38, 1.24, 0.96, 0.79, 0.92, 0.7, 1.12, 0.95, 0.92, 0.8],
    'Pd': [1.87, 1.74, 1.43, 1.39, 1.8, 1.41, 1.43, 1.36, 1.24, 0.92, 0.78, 0.9, 0.69, 1.11, 0.92, 0.9, 0.78],
    'Ag': [1.26, 1.22, 1.03, 1, 1.15, 1.01, 1, 0.95, 1.23, 0.93, 0.8, 0.9, 0.72, 1.09, 0.93, 0.91, 0.82],
    'W': [1.88, 1.68, 1.31, 1.24, 1.82, 1.25, 1.27, 1.19, 1.1, 0.76, 0.68, 0.81, 0.58, 1, 0.78, 0.79, 0.71],
    'Ir': [1.87, 1.57, 1.4, 1.36, 1.8, 1.38, 1.4, 1.33, 1.23, 0.86, 0.74, 0.8, 0.66, 1.08, 0.85, 0.8, 0.74],
    'Pt': [1.88, 1.38, 1.27, 1.2, 1.82, 1.21, 1.23, 1.15, 1, 0.61, 0.44, 0.66, 0.48, 0.86, 0.63, 0.64, 0.49],
    'Au': [1.86, 1.62, 1.37, 1.33, 1.76, 1.37, 1.37, 1.33, 1.13, 0.81, 0.69, 0.77, 0.47, 0.98, 0.79, 0.77, 0.7]
}

target_df = pd.DataFrame(target_data, index=['Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'W', 'Ir', 'Pt', 'Au'])

In [28]:
merged_df['Target'] = merged_df.apply(lambda row: target_df.loc[row['TM1'], row['TM2']], axis=1)

merged_df.head()

,TM1,TM2,edp_1,Hf.ox_1,Nm_1,χ_1,Im_1,r_1,Nd_1,Q_1,...,Hf.ox_2,Nm_2,χ_2,Im_2,r_2,Nd_2,Q_2,ΔGCOOH*_2,ΔGMax_2,Target
0,Ti,Ti,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,-11.39,1.54,0.075540,6.82810,187,2,-1.72,-0.92,1.41,1.30
1,Ti,Cr,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,-6.26,1.66,0.675840,6.76650,189,5,-1.63,-1.15,0.66,1.05
2,Ti,Mn,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,-5.49,1.55,-0.500000,7.43402,197,5,-1.61,-0.50,0.94,1.01
3,Ti,Fe,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,-6.45,1.83,0.153236,7.90240,194,6,-1.49,-1.23,1.14,1.52
4,Ti,Co,10,-11.39,1.54,0.07554,6.8281,187,2,-1.72,...,-5.60,1.88,0.662260,7.88100,192,7,-1.54,-1.77,0.64,1.02


In [29]:
y=merged_df['Target']
y.head()

,Target
0,1.30
1,1.05
2,1.01
3,1.52
4,1.02


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Support Vector Regression (SVR)
svr = SVR()
param_grid_svr = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.2, 0.5]
}
grid_search_svr = GridSearchCV(svr, param_grid_svr, cv=5, scoring='neg_mean_squared_error')
grid_search_svr.fit(X_train, y_train)
best_svr = grid_search_svr.best_estimator_



/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [32]:
# Gaussian Process Regression (GPR)
kernel = C(1.0, (1e-2, 1e2)) * RBF(1, (1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X_train, y_train)



/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [33]:
# Predictions and evaluation
y_pred_svr = best_svr.predict(X_test)
y_pred_gpr = gpr.predict(X_test)

rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
r2_svr = r2_score(y_test, y_pred_svr)

rmse_gpr = np.sqrt(mean_squared_error(y_test, y_pred_gpr))
r2_gpr = r2_score(y_test, y_pred_gpr)

In [35]:
print(f"SVR RMSE: {rmse_svr}, R²: {r2_svr}")
print(f"GPR RMSE: {rmse_gpr}, R²: {r2_gpr}")

SVR RMSE: 0.06331735382811715, R²: 0.9669950014369779
GPR RMSE: 0.01348750459422276, R²: 0.9985023945338615
